<a href="https://colab.research.google.com/github/aee4/MedGemma/blob/main/scripts/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

emmanueleyramagbetor_text_dataset_path = kagglehub.dataset_download('emmanueleyramagbetor/text-dataset')

print('Data source import complete.')


In [ ]:
!pip install "unsloth[colab-new]" torch datasets trl transformers peft accelerate bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 10.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 85.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
import torch
from datasets import load_dataset, DatasetDict
from huggingface_hub import login


In [ ]:
login()

In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import AutoProcessor
from peft import LoraConfig


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-13 19:25:45.861106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763061946.029683      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763061946.076687      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[xformers|WARNING]WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.9.0+cu130 with CUDA 1300 (you have 2.9.0+cu128)
    Python  3.10.19 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.9.0+cu130 with CUDA 1300 (you have 2.9.0+cu128)
    Python  3.10.19 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
import pandas as pd

train = pd.read_csv("/kaggle/input/text-dataset/train.csv")
test = pd.read_csv("/kaggle/input/text-dataset/test.csv")
validation = pd.read_csv("/kaggle/input/text-dataset/validation.csv")


In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch

# Load your CSVs
data_files = {
    "train": "/kaggle/input/text-dataset/train.csv",
    "validation": "/kaggle/input/text-dataset/validation.csv",
    "test": "/kaggle/input/text-dataset/test.csv",
}
dataset = load_dataset("csv", data_files=data_files)

# Inspect
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 2000
    })
})


In [ ]:
#  Extract unique label names safely
label_values = sorted(list(set(dataset['train']['label'])))
formatted_classes = [f"{chr(65 + i)}: {label}" for i, label in enumerate(label_values)]

# Create the prompt for text classification
options = "\n".join(formatted_classes)
PROMPT = f"What is the most likely skin disease described in the text?\n{options}"

# 💬 Format function for text classification
def format_data(example: dict) -> dict:
    label_value = example["label"]
    example["messages"] = [
        {
            "role": "user",
            "content": [{"type": "text", "text": f"{PROMPT}\n\nDescription: {example['text']}"}],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": str(label_value)}],
        },
    ]
    return example

# 🧠 Apply formatting to all splits
formatted_data = dataset.map(format_data, remove_columns=["label"])

print("\n✅ Dataset is formatted for training.")
formatted_data

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]


✅ Dataset is formatted for training.


DatasetDict({
    train: Dataset({
        features: ['disease', 'text', 'messages'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['disease', 'text', 'messages'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['disease', 'text', 'messages'],
        num_rows: 2000
    })
})

In [ ]:
model_id = "google/medgemma-4b-it"
model, processor = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=2048,
    dtype=None,             # Unsloth handles dtype automatically
    load_in_4bit=True,      # This enables 4-bit quantization
)
processor.tokenizer.padding_side = "right"
print("Model loaded with Unsloth optimizations.")

==((====))==  Unsloth 2025.11.2: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.12G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Model loaded with Unsloth optimizations.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules="all-linear",
    use_gradient_checkpointing=True,
)

def collate_fn(examples: list[dict]):
    images = [example['image'] for example in examples]
    texts = [
        processor.apply_chat_template(ex['messages'], add_generation_prompt=False, tokenize=False).strip()
        for ex in examples
    ]
    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)
    labels = batch['input_ids'].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    batch["labels"] = labels
    return batch

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [ ]:
from trl import SFTTrainer, SFTConfig
import torch

In [ ]:
# 1️⃣ Define the formatting function
def format_dataset_for_unsloth(example):
    text = processor.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}


# 2️⃣ Format the entire dataset (keep num_proc=1 for safety)
preformatted_dataset = formatted_data.map(
    format_dataset_for_unsloth,
    num_proc=1
)

print("✅ Dataset pre-formatted with a 'text' column.")
print("Example entry:\n", preformatted_dataset["train"][0]["text"])


# 3️⃣ Define training arguments — optimized for Colab
args = SFTConfig(
    output_dir="medgemma-unsloth",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,     # balances speed & stability
    warmup_steps=5,
    learning_rate=2e-4,
    fp16=True,                         # mixed precision for memory efficiency
    logging_steps=10,
    optim="adamw_torch",
    save_strategy="epoch",
    eval_strategy="epoch",
    save_total_limit=1,
    seed=42,
    report_to="none",                  # disable wandb
)


# 4️⃣ Prepare train/validation splits
train_data = preformatted_dataset["train"]
val_data = preformatted_dataset["validation"]


# 5️⃣ Initialize the trainer with reduced sequence length
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    dataset_text_field="text",
    max_seq_length=1024,               # lower seq length = less memory
    dataset_num_proc=1,
    args=args,
)

print("🚀 Trainer ready. Training for 3 epochs with evaluation after each epoch.")


Map (num_proc=1):   0%|          | 0/3000 [00:00<?, ? examples/s]

Map (num_proc=1):   0%|          | 0/2000 [00:00<?, ? examples/s]

Map (num_proc=1):   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Dataset pre-formatted with a 'text' column.
Example entry:
 <bos><start_of_turn>user
What is the most likely skin disease described in the text?
A: 0
B: 1

Description: Blistering or weeping lesions in acute flares appearing in skin creases and folds. The patient feels unbearably itchy, especially at night and describes the disease as mild and intermittent. Sometimes flares after infections or insect bites.<end_of_turn>
<start_of_turn>model
1<end_of_turn>

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/3000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/2000 [00:00<?, ? examples/s]

🚀 Trainer ready. Training for 3 epochs with evaluation after each epoch.


In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 10 | Total steps = 15,000
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 38,497,792 of 4,338,577,264 (0.89% trained)


Epoch,Training Loss,Validation Loss
1,0.087000,0.170258
2,0.087700,0.169756
3,0.085500,0.166481
4,0.088300,0.162906
5,0.085500,0.164516
6,0.082900,0.167111
7,0.082400,0.165915
8,0.081800,0.162073
9,0.077400,0.162008
10,0.075800,0.157538


Unsloth: Not an error, but Gemma3ForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=15000, training_loss=0.0860549313823382, metrics={'train_runtime': 20487.4306, 'train_samples_per_second': 1.464, 'train_steps_per_second': 0.732, 'total_flos': 4.78541474621952e+16, 'train_loss': 0.0860549313823382, 'epoch': 10.0})